In [1]:
import os

# Define dataset root path
dataset_path = r"C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN"

# Define processed image paths
processed_folder = os.path.join(dataset_path, "Processed_Images")
image_folder = os.path.join(processed_folder, "images")
mask_folder = os.path.join(processed_folder, "masks")

# ✅ Create necessary directories if they don’t exist
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(image_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

# ✅ Verify dataset structure
folders = ["TRAIN", "VAL", "TEST"]
for folder in folders:
    folder_path = os.path.join(dataset_path, folder)
    if os.path.exists(folder_path):
        print(f"✅ Found: {folder}")
    else:
        print(f"❌ Missing: {folder}")

print("✅ Dataset structure verified. Now ready for image processing!")


✅ Found: TRAIN
✅ Found: VAL
✅ Found: TEST
✅ Dataset structure verified. Now ready for image processing!


In [6]:
import os
import numpy as np
import cv2
import json
from tensorflow.keras.preprocessing.image import save_img

# ✅ Define dataset directories
dataset_path = r"C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN"
processed_folder = os.path.join(dataset_path, "Processed_Images")
original_folder = os.path.join(processed_folder, "original")
overlay_folder = os.path.join(processed_folder, "overlay")

# ✅ Create folders if they don’t exist
os.makedirs(original_folder, exist_ok=True)
os.makedirs(overlay_folder, exist_ok=True)

# ✅ Load annotation JSON
annotation_file = os.path.join(dataset_path, "annotations_all.json")

with open(annotation_file, "r") as f:
    annotations = json.load(f)

# ✅ Function to create mask from annotation
def create_mask_from_annotation(annotation, original_size):
    h_orig, w_orig = original_size  
    mask = np.zeros((h_orig, w_orig), dtype=np.uint8)
    contour_list = []

    for region in annotation["regions"]:
        shape_attr = region["shape_attributes"]
        
        if shape_attr["name"] == "polygon":
            all_x = np.array(shape_attr["all_points_x"])
            all_y = np.array(shape_attr["all_points_y"])

            all_x = np.clip(all_x, 0, w_orig - 1)
            all_y = np.clip(all_y, 0, h_orig - 1)

            contour = np.array(list(zip(all_x, all_y)), dtype=np.int32)
            contour_list.append(contour)

            # ✅ Draw the tumor mask properly
            cv2.fillPoly(mask, [contour], 255)

    return mask, contour_list

# ✅ Process all images from TRAIN, TEST, and VAL folders
for folder in ["TRAIN", "TEST", "VAL"]:
    folder_path = os.path.join(dataset_path, folder)

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

        # ✅ Check if it's a valid image
        if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️ Skipping {img_name}, unable to read.")
            continue

        original_size = img.shape[:2]  

        # ✅ Save original image
        save_img(os.path.join(original_folder, img_name), np.expand_dims(img, axis=-1))

        base_filename = os.path.splitext(img_name)[0]  
        matching_key = next((key for key in annotations.keys() if base_filename in key), None)

        if matching_key:
            mask, contours = create_mask_from_annotation(annotations[matching_key], original_size)

            # ✅ Fix color formatting
            img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

            # ✅ Create mask overlay
            colored_mask = np.zeros_like(img_color)
            colored_mask[:, :, 2] = mask  

            # ✅ Blend overlay
            overlayed_image = cv2.addWeighted(img_color, 1, colored_mask, 0.6, 0)

            # ✅ Draw tumor boundary in **Blue**
            cv2.polylines(overlayed_image, contours, isClosed=True, color=(0, 0, 255), thickness=2)

            # ✅ Add tumor label at correct position
            if len(contours) > 0:
                M = cv2.moments(contours[0])
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    cv2.putText(overlayed_image, "Tumor Detected", (cX - 10, cY - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 100, 10), 2, cv2.LINE_AA)

            # ✅ Save overlay image
            save_img(os.path.join(overlay_folder, img_name), overlayed_image)

print(f"✅ All original images saved in: {original_folder}")
print(f"✅ All overlay images saved in: {overlay_folder}")


✅ All original images saved in: C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\original
✅ All overlay images saved in: C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\overlay


Processing All Images (Original + Overlayed)

In [14]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import save_img

# ✅ Define dataset directories
processed_folder = os.path.join(dataset_path, "Processed_Images")
original_folder = os.path.join(processed_folder, "original")
overlay_folder = os.path.join(processed_folder, "overlay")
final_original_folder = os.path.join(processed_folder, "final_original")
final_overlay_folder = os.path.join(processed_folder, "final_overlay")

# ✅ Create necessary directories
os.makedirs(final_original_folder, exist_ok=True)
os.makedirs(final_overlay_folder, exist_ok=True)

# ✅ Target size (256x256) - Ensuring consistency
target_size = (256, 256)

# ✅ Function to resize both images while preserving alignment
def resize_pair(image, overlay, target_size):
    image_resized = cv2.resize(image, target_size, interpolation=cv2.INTER_NEAREST)
    overlay_resized = cv2.resize(overlay, target_size, interpolation=cv2.INTER_NEAREST)
    return image_resized, overlay_resized

# ✅ Process all images in the original and overlay folders
for img_name in os.listdir(original_folder):
    img_path = os.path.join(original_folder, img_name)
    overlay_path = os.path.join(overlay_folder, img_name)

    # ✅ Load images
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    overlay = cv2.imread(overlay_path, cv2.IMREAD_GRAYSCALE)

    if img is None or overlay is None:
        print(f"⚠️ Skipping {img_name}, unable to read.")
        continue

    # ✅ Resize both images together
    img_resized, overlay_resized = resize_pair(img, overlay, target_size)

    # ✅ Save resized images
    save_img(os.path.join(final_original_folder, img_name), np.expand_dims(img_resized, axis=-1))
    save_img(os.path.join(final_overlay_folder, img_name), np.expand_dims(overlay_resized, axis=-1))

print(f"✅ All resized original images saved in: {final_original_folder}")
print(f"✅ All resized overlay images saved in: {final_overlay_folder}")


✅ All resized original images saved in: C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\final_original
✅ All resized overlay images saved in: C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\final_overlay


In [20]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import save_img

# ✅ Define dataset directories
base_folder = r"C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN"
processed_folder = os.path.join(base_folder, "Processed_Images")

# ✅ Define paths for resized images
resized_original_folder = os.path.join(processed_folder, "resized_original")
resized_overlays_folder = os.path.join(processed_folder, "resized_overlays")

# ✅ Define paths for final processed images
final_images_folder = os.path.join(processed_folder, "final_images")
final_original_folder = os.path.join(final_images_folder, "images_resized_original")
final_masked_folder = os.path.join(final_images_folder, "images_masked_overlays")

# ✅ Create necessary directories
os.makedirs(resized_original_folder, exist_ok=True)
os.makedirs(resized_overlays_folder, exist_ok=True)
os.makedirs(final_original_folder, exist_ok=True)
os.makedirs(final_masked_folder, exist_ok=True)

# ✅ Target size for all images
img_size = (256, 256)

# ✅ Function to resize images
def resize_image(image_path, output_path, img_size, is_gray=True):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE if is_gray else cv2.IMREAD_COLOR)
    if img is not None:
        resized_img = cv2.resize(img, img_size)
        save_img(output_path, np.expand_dims(resized_img, axis=-1) if is_gray else resized_img)
    else:
        print(f"⚠️ Warning: Unable to load {image_path}")

# ✅ Resize original images
original_folder = os.path.join(processed_folder, "original")
for img_name in os.listdir(original_folder):
    input_path = os.path.join(original_folder, img_name)
    output_path = os.path.join(resized_original_folder, img_name)
    resize_image(input_path, output_path, img_size, is_gray=True)

# ✅ Resize overlay images
overlay_folder = os.path.join(processed_folder, "overlay")
for img_name in os.listdir(overlay_folder):
    input_path = os.path.join(overlay_folder, img_name)
    output_path = os.path.join(resized_overlays_folder, img_name)
    resize_image(input_path, output_path, img_size, is_gray=False)

print(f"✅ All images resized and stored in:")
print(f"   📂 {resized_original_folder}")
print(f"   📂 {resized_overlays_folder}")


✅ All images resized and stored in:
   📂 C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\resized_original
   📂 C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\resized_overlays


In [28]:
import os
import numpy as np
import cv2
import json
from tensorflow.keras.preprocessing.image import save_img

# ✅ Define dataset directories
base_folder = r"C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN"
processed_folder = os.path.join(base_folder, "Processed_Images")

# ✅ Paths for images and masks
original_folder = os.path.join(processed_folder, "original")  # Use ORIGINAL images
masks_folder = os.path.join(processed_folder, "masks")  # ✅ Binary masks will be stored here

# ✅ Create necessary directories
os.makedirs(masks_folder, exist_ok=True)

# ✅ Load annotation JSON
annotation_file = os.path.join(base_folder, "annotations_all.json")
with open(annotation_file, "r") as f:
    annotations = json.load(f)

# ✅ Function to create mask from annotation
def create_mask_from_annotation(annotation, img_shape):
    h_orig, w_orig = img_shape  # Get original image dimensions
    mask = np.zeros((h_orig, w_orig), dtype=np.uint8)  # Black background

    for region in annotation["regions"]:
        shape_attr = region["shape_attributes"]
        
        if shape_attr["name"] == "polygon":
            all_x = np.array(shape_attr["all_points_x"])
            all_y = np.array(shape_attr["all_points_y"])

            # Ensure coordinates stay within the image bounds
            all_x = np.clip(all_x, 0, w_orig - 1)
            all_y = np.clip(all_y, 0, h_orig - 1)

            # Convert points to contour format
            contour = np.array(list(zip(all_x, all_y)), dtype=np.int32)

            # ✅ Fill the tumor region with white (255)
            cv2.fillPoly(mask, [contour], 255)

    return mask

# ✅ Process all images from the original folder
for img_name in os.listdir(original_folder):
    img_path = os.path.join(original_folder, img_name)
    mask_path = os.path.join(masks_folder, img_name)

    # ✅ Load original image to get dimensions
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        print(f"⚠️ Skipping {img_name}, unable to read.")
        continue

    # ✅ Extract corresponding annotation key
    base_filename = os.path.splitext(img_name)[0]  
    matching_key = next((key for key in annotations.keys() if base_filename in key), None)

    if matching_key:
        # ✅ Generate tumor mask
        mask = create_mask_from_annotation(annotations[matching_key], img.shape)

        # ✅ Save the generated binary mask
        save_img(mask_path, np.expand_dims(mask, axis=-1))

print(f"✅ All binary masks generated and saved in: {masks_folder}")
# isse masking perfect ho raha hai with original size images ab bus isko resize krna hai


✅ All binary masks generated and saved in: C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\masks


In [30]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import save_img

# ✅ Paths for resized masks
resized_masks_folder = os.path.join(processed_folder, "resized_masks")  # ✅ Final aligned masks
os.makedirs(resized_masks_folder, exist_ok=True)

# ✅ Target size
target_size = (256, 256)

# ✅ Function to resize while preserving aspect ratio
def resize_with_aspect_ratio(image, target_size, interpolation=cv2.INTER_NEAREST):
    h, w = image.shape[:2]
    target_w, target_h = target_size

    # ✅ Compute scale to maintain aspect ratio
    scale = min(target_w / w, target_h / h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    # ✅ Resize image
    resized = cv2.resize(image, (new_w, new_h), interpolation=interpolation)

    # ✅ Create a black canvas
    final_image = np.zeros((target_h, target_w), dtype=np.uint8)

    # ✅ Center the resized image on the black canvas
    x_offset = (target_w - new_w) // 2
    y_offset = (target_h - new_h) // 2
    final_image[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

    return final_image

# ✅ Resize masks while preserving aspect ratio
masks_folder = os.path.join(processed_folder, "masks")  # Previously generated masks

for mask_name in os.listdir(masks_folder):
    mask_path = os.path.join(masks_folder, mask_name)
    resized_mask_path = os.path.join(resized_masks_folder, mask_name)

    # ✅ Load mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if mask is not None:
        # ✅ Resize mask while maintaining aspect ratio
        resized_mask = resize_with_aspect_ratio(mask, target_size)

        # ✅ Save resized mask
        save_img(resized_mask_path, np.expand_dims(resized_mask, axis=-1))
    else:
        print(f"⚠️ Warning: Unable to read {mask_name}")

print(f"✅ All resized masks stored in: {resized_masks_folder}")
# 256 by 256 pixels me properly resize ho chuka hai wo bhi perfectly aligned


✅ All resized masks stored in: C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\resized_masks


In [31]:
import os
import numpy as np
import cv2
import albumentations as A
from tensorflow.keras.preprocessing.image import save_img

# ✅ Define dataset directories
processed_folder = r"C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images"
resized_original_folder = os.path.join(processed_folder, "resized_original")
resized_mask_folder = os.path.join(processed_folder, "resized_masks")  # Binary Masks
augmented_folder = os.path.join(processed_folder, "augmented_images")
augmented_mask_folder = os.path.join(processed_folder, "augmented_masks")

# ✅ Create necessary directories
os.makedirs(augmented_folder, exist_ok=True)
os.makedirs(augmented_mask_folder, exist_ok=True)

# ✅ Augmentation Pipeline (Ensuring tumor visibility)
augment_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),  
    A.VerticalFlip(p=0.5),  
    A.RandomRotate90(p=0.5),  
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),  
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),  # Improve contrast
    A.CLAHE(clip_limit=2, tile_grid_size=(8, 8), p=0.3),  # Local histogram equalization
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
    A.OneOf([
        A.RandomGamma(gamma_limit=(80, 120), p=0.4),  # Adjust gamma
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1, p=0.3)  # Slight color change
    ], p=0.3)
], additional_targets={'mask': 'mask'})  # Ensure mask follows the same transformations

# ✅ Function to Augment Image-Mask Pairs
def augment_and_save(img, mask, img_name):
    augmented = augment_pipeline(image=img, mask=mask)
    aug_img, aug_mask = augmented["image"], augmented["mask"]

    # ✅ Save Augmented Images
    save_img(os.path.join(augmented_folder, f"aug_{img_name}"), np.expand_dims(aug_img, axis=-1))
    save_img(os.path.join(augmented_mask_folder, f"aug_{img_name}"), np.expand_dims(aug_mask, axis=-1))

# ✅ Apply augmentation to all images and masks
for img_name in os.listdir(resized_original_folder):
    img_path = os.path.join(resized_original_folder, img_name)
    mask_path = os.path.join(resized_mask_folder, img_name)

    # ✅ Load image and mask
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if img is None or mask is None:
        print(f"⚠️ Skipping {img_name}, unable to read.")
        continue

    # ✅ Perform augmentation and save
    augment_and_save(img, mask, img_name)

print(f"✅ Augmentation complete! Augmented images and masks saved in:")
print(f"   📂 {augmented_folder}")
print(f"   📂 {augmented_mask_folder}")


C:\Users\Suraj Yadav\AppData\Local\Temp\ipykernel_14772\2088412016.py:26: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),


✅ Augmentation complete! Augmented images and masks saved in:
   📂 C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\augmented_images
   📂 C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images\augmented_masks


In [32]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

# ✅ Define dataset paths
processed_folder = r"C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images"
augmented_images_folder = os.path.join(processed_folder, "augmented_images")
augmented_masks_folder = os.path.join(processed_folder, "augmented_masks")

# ✅ Define paths for train, val, and test datasets
train_folder = os.path.join(processed_folder, "train")
val_folder = os.path.join(processed_folder, "val")
test_folder = os.path.join(processed_folder, "test")

# ✅ Create subdirectories for images and masks
for split in [train_folder, val_folder, test_folder]:
    os.makedirs(os.path.join(split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split, "masks"), exist_ok=True)

# ✅ Get list of augmented image files
image_files = sorted(os.listdir(augmented_images_folder))

# ✅ Ensure masks follow the correct naming pattern
mask_files = sorted(os.listdir(augmented_masks_folder))

# ✅ Ensure we only work with matching image-mask pairs
image_files = [f for f in image_files if f in mask_files]

# ✅ Split into train (80%), val (10%), and test (10%)
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)

# ✅ Function to move files
def move_files(file_list, src_img_folder, src_mask_folder, dest_folder):
    for file_name in file_list:
        img_src = os.path.join(src_img_folder, file_name)
        mask_src = os.path.join(src_mask_folder, file_name)

        img_dest = os.path.join(dest_folder, "images", file_name)
        mask_dest = os.path.join(dest_folder, "masks", file_name)

        # ✅ Check if files exist before copying
        if not os.path.exists(img_src):
            print(f"⚠️ Missing Image: {img_src}")
            continue
        if not os.path.exists(mask_src):
            print(f"⚠️ Missing Mask: {mask_src}")
            continue

        shutil.copy(img_src, img_dest)
        shutil.copy(mask_src, mask_dest)

# ✅ Move files to respective sets
move_files(train_files, augmented_images_folder, augmented_masks_folder, train_folder)
move_files(val_files, augmented_images_folder, augmented_masks_folder, val_folder)
move_files(test_files, augmented_images_folder, augmented_masks_folder, test_folder)

# ✅ Verify dataset split
train_img_count = len(os.listdir(os.path.join(train_folder, "images")))
val_img_count = len(os.listdir(os.path.join(val_folder, "images")))
test_img_count = len(os.listdir(os.path.join(test_folder, "images")))

print(f"✅ Train dataset: {train_img_count} images")
print(f"✅ Validation dataset: {val_img_count} images")
print(f"✅ Test dataset: {test_img_count} images")
print(f"✅ Dataset split complete!")


✅ Train dataset: 640 images
✅ Validation dataset: 80 images
✅ Test dataset: 81 images
✅ Dataset split complete!


In [33]:
import os
import numpy as np
import cv2
import tensorflow as tf

# ✅ Define dataset directories
processed_folder = r"C:\Users\Suraj Yadav\FinalProjectTest\datasetMRI\Br35H-Mask-RCNN\Processed_Images"

train_images_path = os.path.join(processed_folder, "train/images")
train_masks_path = os.path.join(processed_folder, "train/masks")

val_images_path = os.path.join(processed_folder, "val/images")
val_masks_path = os.path.join(processed_folder, "val/masks")

test_images_path = os.path.join(processed_folder, "test/images")
test_masks_path = os.path.join(processed_folder, "test/masks")

# ✅ Function to Load Images & Masks
def load_data(image_folder, mask_folder):
    image_filenames = sorted(os.listdir(image_folder))
    mask_filenames = sorted(os.listdir(mask_folder))

    images, masks = [], []
    for img_file, mask_file in zip(image_filenames, mask_filenames):
        img_path = os.path.join(image_folder, img_file)
        mask_path = os.path.join(mask_folder, mask_file)

        # ✅ Load images
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) / 255.0  # Normalize
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) / 255.0  # Normalize

        # ✅ Expand dimensions for model input shape (256,256,1)
        img = np.expand_dims(img, axis=-1)
        mask = np.expand_dims(mask, axis=-1)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

# ✅ Load Train, Val, Test Data
X_train, Y_train = load_data(train_images_path, train_masks_path)
X_val, Y_val = load_data(val_images_path, val_masks_path)
X_test, Y_test = load_data(test_images_path, test_masks_path)

# ✅ Print dataset shapes
print(f"✅ Dataset Loaded:")
print(f"   - Train: {X_train.shape}, {Y_train.shape}")
print(f"   - Validation: {X_val.shape}, {Y_val.shape}")
print(f"   - Test: {X_test.shape}, {Y_test.shape}")


✅ Dataset Loaded:
   - Train: (640, 256, 256, 1), (640, 256, 256, 1)
   - Validation: (80, 256, 256, 1), (80, 256, 256, 1)
   - Test: (81, 256, 256, 1), (81, 256, 256, 1)


In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

# ✅ Function to Build U-Net Model
def build_unet(input_shape=(256, 256, 1)):
    inputs = Input(input_shape)

    # Encoder (Contracting Path)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder (Expanding Path)
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs, outputs)
    return model

# ✅ Initialize U-Net Model
unet_model = build_unet()
unet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.MeanIoU(num_classes=2)]
)

# ✅ Print Model Summary
unet_model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

Splitting Dataset into Train/Val/Test